Импортируем библиотеки

In [ ]:
!pip install catboost

Это помогло с ошибкой ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
pip install --upgrade --force-reinstall pip setuptools wheel

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.45.1
    Uninstalling wheel-0.45.1:
      Successfully uninstalled wheel-0.45.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
import warnings
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
import seaborn as sns
from sklearn.preprocessing import TargetEncoder
import sklearn
from catboost import CatBoostClassifier
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import plotly.graph_objs as go
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

Мы решаем задачу предсказания непрерывного значения заработной платы в России за 2024 преимущественно на данных вакансий Москвы

Медианные зарплаты по районам Москвы и вне Москвы варьируются от 100 до 150 тысяч рублей, всего имеется 161 уникальная профессия, а индустрии всего 3:  'Электроника, приборостроение, бытовая техника, компьютеры и оргтехника' , 'Автомобильный бизнес', 'Перевозки, логистика, склад, ВЭД'

**EDA**:

Выписываем категориальные фичи и фичи, которые станут категориальными после преобразования датасета

In [ ]:
cat_features = ['premium', 'has_test', 'response_letter_required', 'archived', 'schedule', 'accept_temporary', 'accept_incomplete_resumes',
                'experience', 'employment', 'is_adv_vacancy' ,'employer_trusted', 'employer_type', 'metro_feature', 'has_email', 'industry', 'AO', 'exp_loc_category']

Считываем данные

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/vacancies.csv')
data

,premium,has_test,response_letter_required,archived,schedule,accept_temporary,professional_roles,accept_incomplete_resumes,experience,employment,...,has_email,phones_count,vacancy_type,lat,lon,industry,stations_within_km,distance_to_the_nearest(m),AO,salary
0,False,False,False,True,Полный день,True,Монтажник,False,От 1 года до 3 лет,Полная занятость,...,False,0,Открытая,55.850439,37.669172,"Электроника, приборостроение, бытовая техника,...",0.0,1139.547085,СВАО,100000.0
1,False,False,False,False,Полный день,False,Бухгалтер,False,От 3 до 6 лет,Полная занятость,...,False,0,Открытая,55.758762,37.750218,"Электроника, приборостроение, бытовая техника,...",2.0,109.992781,ВАО,75000.0
2,False,False,False,False,Полный день,False,Технический директор (CTO),False,Более 6 лет,Полная занятость,...,False,0,Открытая,55.743236,37.650630,"Электроника, приборостроение, бытовая техника,...",4.0,193.529616,ЦАО,282750.0
3,False,False,False,False,Полный день,False,Бухгалтер,False,От 3 до 6 лет,Полная занятость,...,False,0,Открытая,55.758762,37.750218,"Электроника, приборостроение, бытовая техника,...",2.0,109.992781,ВАО,75000.0
4,False,False,False,False,Полный день,False,Руководитель отдела продаж,False,От 3 до 6 лет,Полная занятость,...,False,0,Анонимная,NaN,NaN,"Электроника, приборостроение, бытовая техника,...",NaN,NaN,Не в Москве,304500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31121,False,False,False,False,Полный день,False,Другое,True,От 1 года до 3 лет,Полная занятость,...,False,0,Открытая,55.695144,37.429537,"Перевозки, логистика, склад, ВЭД",0.0,1878.403281,ЗАО,87000.0
31122,False,False,False,False,Сменный график,False,"Оператор ПК, оператор базы данных",True,Нет опыта,Полная занятость,...,False,0,Открытая,55.748057,37.587542,"Перевозки, логистика, склад, ВЭД",3.0,237.436414,ЦАО,47850.0
31123,False,False,False,False,Полный день,False,"Менеджер по продажам, менеджер по работе с кли...",False,От 1 года до 3 лет,Полная занятость,...,False,0,Открытая,55.719166,37.629241,"Перевозки, логистика, склад, ВЭД",1.0,867.808517,ЮАО,120000.0
31124,False,False,False,False,Полный день,False,"Менеджер по продажам, менеджер по работе с кли...",False,Нет опыта,Полная занятость,...,False,0,Открытая,55.716392,37.676214,"Перевозки, логистика, склад, ВЭД",2.0,186.815416,ЮВАО,43587.0


Избавляемся от выбросов

In [ ]:
data = data.drop(data[data['salary'] > 800000].index.to_list(), axis = 0)

Настолько большие зарплаты выбиваются из общего списка и их можно удалить, чтобы они не мешали модели находить закономерности, а так же таких объектов всего 8, что составляет около 0.025 от процента данных:

In [ ]:
data[ data['salary'] > 800000 ]['salary'].sort_values(ascending=False).count()

0

Преобразуем фичи

Отдельно хочется сказать про занятость и работодателя. Параметр занятости увеличивает заработок, если она полная, иначе зарплата зарплата меньше.

Если в колонке про работодателя стоит 'company', то обычно необходимо проходить собеседования или удовлетворять каким либо специфичным требованиям, что влияет на зарплату. В других случаях нет необходимости рахличать работодателя.

In [ ]:
#FEATURE ENGINERING

#stations_within_km, distance_to_the_nearest(m) 	объедияем
data['metro_feature'] = 'Никаких станций нет'

data.loc[data['distance_to_the_nearest(m)'] > 1000, 'metro_feature'] = 'Станция далеко, но есть'

data.loc[data['stations_within_km'] > 0, 'metro_feature'] = 'Станция рядом есть'

#полная/неполная занятость:
data.loc[data["employment"] != "Полная занятость", "employment"] = "Не полная занятость"

#работодатель:
data.loc[data["employer_type"] != "company", "employer_type"] = "not a company"

#добавляю категорию включающую опыт+локацию:
moscow_regions = ['свао', 'вао', 'цао', 'юзао', 'сзао', 'сао', 'ювао', 'новомосковский', 'зао', 'юао', 'троицкий', 'зелао']
data['AO_loc'] = data['AO'].apply(
    lambda x: 'Москва' if x in moscow_regions else 'Не в Москве'
)

exp_values = data['experience'].unique()

exp_categories = {exp_value: i+1 for i, exp_value in enumerate(exp_values)}

data['exp_cat'] = data['experience'].map(exp_categories)

ao_rules = {
    ('Москва', 1): 1,
    ('Москва', 2): 2,
    ('Москва', 3): 3,
    ('Москва', 4): 4,
    ('Не в Москве', 1): 5,
    ('Не в Москве', 2): 6,
    ('Не в Москве', 3): 7,
    ('Не в Москве', 4): 8
}

#создадим новую колонку с финальной категорией
data['exp_loc_category'] = data.apply(lambda row: ao_rules[(row['AO_loc'], row['exp_cat'])], axis=1)

Удаление лишних колонок и косметическое преобразование датасета

In [ ]:
#дропаем ненужные фичи
data = data.drop(['lat','lon','stations_within_km', 'distance_to_the_nearest(m)', 'vacancy_type', 'exp_cat', 'AO_loc'], axis = 1)
data = data.dropna()

#нижний регистр
columns_to_lower = ['schedule', 'professional_roles', 'experience', 'employment', 'industry', 'AO']
for column in columns_to_lower:
    data[column] = data[column].astype(str).str.lower()


#возвращаем в категориальную фичу
def type_converter(X):
    for i in cat_features:
        X[i] = X[i].astype('category')
    return X
data = type_converter(data)

Разделение данных на train, val, test, применение Target encoder (кодировка как медианная зарплата для каждой профессии) для категории профессии и к ней же Standard scaler из-за масштаба зарплат

Прологарифмируем зарплаты, чтобы уменьшить масштаб зарплат, и разобьем зарплаты на 5 категорий по квантилям, чтобы не было дисбаланса классов

Делим данные

In [ ]:
#разделение данных
X = data.drop('salary', axis = 1)
y = data['salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, shuffle=True, random_state=42)

Применяем label encoder и standard scaler

In [ ]:
#label encoder
encoder_dict = {}
for col in cat_features:
    encoder_dict[col] = preprocessing.LabelEncoder().fit(X_train[[col]])
    X_train[col] = encoder_dict[col].transform(X_train[[col]])
    X_test[col] = encoder_dict[col].transform(X_test[[col]])
    X_val[col] = encoder_dict[col].transform(X_val[[col]])

X_train['professional_roles'] = X_train['professional_roles'].astype(str)
X_test['professional_roles'] = X_test['professional_roles'].astype(str)
X_val['professional_roles'] = X_val['professional_roles'].astype(str)

# Применение TargetEncoder
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    encoder = TargetEncoder(categories = 'auto', smooth='auto')
    encoder.fit(X_train[['professional_roles']], y_train)
    X_train['professional_roles'] = encoder.transform(X_train[['professional_roles']])
    X_test['professional_roles'] = encoder.transform(X_test[['professional_roles']])
    X_val['professional_roles'] = encoder.transform(X_val[['professional_roles']])

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    scaler = preprocessing.StandardScaler().fit(X_train[['professional_roles']])
    X_train['professional_roles'] = scaler.transform(X_train[['professional_roles']])
    X_test['professional_roles'] = scaler.transform(X_test[['professional_roles']])
    X_val['professional_roles'] = scaler.transform(X_val[['professional_roles']])

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was 

In [ ]:
y_train = np.log(y_train.values.reshape(-1, 1)).flatten()
y_test = np.log(y_test.values.reshape(-1, 1)).flatten()
y_val = np.log(y_val.values.reshape(-1, 1)).flatten()

#сохраняем исходные зарплаты, до перевода в категории
y_train_original = y_train.copy()
y_test_original = y_test.copy()
y_val_original = y_val.copy()

_, bins = pd.qcut(y_train, q=5, retbins=True, labels=False)

#разбиваем на квантили
y_train = pd.cut(y_train, bins=bins, labels=False, include_lowest=True)
y_test = pd.cut(y_test, bins=bins, labels=False, include_lowest=True)
y_val = pd.cut(y_val, bins=bins, labels=False, include_lowest=True)

#переводим нужные переменные в категории
def type_converter(X):
    for i in cat_features:
        X[i] = X[i].astype('category')
    return X
X_train = type_converter(X_train)
X_test = type_converter(X_test)
X_val = type_converter(X_val)

**FIT И ОЦЕНКА ПРОМЕЖУТОЧНЫХ РЕЗУЛЬТАТОВ МОДЕЛЕЙ**

Обучим классификатор catboost для предсказания метки класса зарплаты и установим early_stopping_rounds, чтобы остановить обучение, когда изменение метрики будет малым

In [ ]:
cbс = CatBoostClassifier(
    n_estimators=2000,
    learning_rate=1,
    max_depth=5,

    cat_features=cat_features,
    random_state=42,
    eval_metric='MultiClass',
    loss_function='MultiClass',
    early_stopping_rounds=50,
    verbose=100
)
cbс.fit(X_train, y_train, eval_set = (X_val, y_val))


0:	learn: 1.3710738	test: 1.3658168	best: 1.3658168 (0)	total: 366ms	remaining: 12m 12s
100:	learn: 1.0073269	test: 1.0291641	best: 1.0291641 (100)	total: 41.6s	remaining: 13m 2s
200:	learn: 0.8818152	test: 0.9450251	best: 0.9450251 (200)	total: 1m 21s	remaining: 12m 13s
300:	learn: 0.7844104	test: 0.8824030	best: 0.8824030 (300)	total: 1m 58s	remaining: 11m 7s
400:	learn: 0.7045088	test: 0.8361311	best: 0.8361311 (400)	total: 2m 36s	remaining: 10m 24s
500:	learn: 0.6395183	test: 0.8016870	best: 0.8016870 (500)	total: 3m 11s	remaining: 9m 34s
600:	learn: 0.5824405	test: 0.7785605	best: 0.7785605 (600)	total: 3m 53s	remaining: 9m 2s
700:	learn: 0.5352242	test: 0.7544809	best: 0.7543771 (699)	total: 4m 33s	remaining: 8m 27s
800:	learn: 0.4934156	test: 0.7311056	best: 0.7311056 (800)	total: 5m 10s	remaining: 7m 45s
900:	learn: 0.4562066	test: 0.7118030	best: 0.7118030 (900)	total: 5m 45s	remaining: 7m 1s
1000:	learn: 0.4226567	test: 0.6970655	best: 0.6970655 (1000)	total: 6m 24s	remaining

Получаем предсказания классов (y_pred) и вероятности классов (y_proba) и выводим метрики качества (roc-auc работает с помощью голосования большинства)

In [ ]:
y_pred_val = cbс.predict(X_val)
y_pred_train = cbс.predict(X_train)
y_pred_test = cbс.predict(X_test)
y_proba_train = cbс.predict_proba(X_train)
y_proba_val = cbс.predict_proba(X_val)
y_proba_test = cbс.predict_proba(X_test)


print("train accuracy score:", accuracy_score(y_train, y_pred_train))
print("val accuracy score:", accuracy_score(y_val, y_pred_val))
print("test accuracy score:", accuracy_score(y_test, y_pred_test))

print("train ROC-AUC score:", roc_auc_score(y_train, y_proba_train, multi_class='ovo'))
print("val ROC-AUC score:", roc_auc_score(y_val, y_proba_val, multi_class='ovo'))
print("test ROC-AUC score:", roc_auc_score(y_test, y_proba_test, multi_class='ovo'))

train accuracy score: 0.7544721998388396
val accuracy score: 0.7567201701798492
test accuracy score: 0.6982530780635596
train ROC-AUC score: 0.9449289756406806
val ROC-AUC score: 0.9459893021615681
test ROC-AUC score: 0.9199013014096394


Для мультиклассовой (5 классов) задачи accuracy 0.7 на тестовой выборке в нашем случае не плох, учитывая специфику задачи (ошибка в предсказании класса зарплаты не так критична, как, например, в медицине или финансах, где такой accuracy мог бы быть не достаточным ) и то, что баланс классов сохраняется и accuracy из-за этого не искажен. ROC-AUC, подсчитанный как среднее для всех классов (one vs one), показывает неплохие значения

Вычисляем roc-auc для каждого класса, чтобы понять, как модель справляется с предсказаниями каждого класса по отдельности

In [ ]:

y_val = np.array(y_val)

y_proba_val = np.array(y_proba_val)  # np.array, если ещё не массив

# Бинаризуем метки классов (one-hot encoding)
y_val_bin = label_binarize(y_val, classes=np.unique(y_val))

# Вычисляем ROC AUC для каждого класса отдельно
for i, class_label in enumerate(np.unique(y_val)):
    auc_score = roc_auc_score(y_val_bin[:, i], y_proba_val[:, i])
    print(f"ROC AUC для класса {class_label}: {auc_score:.4f}")

ROC AUC для класса 0: 0.9746
ROC AUC для класса 1: 0.9409
ROC AUC для класса 2: 0.9228
ROC AUC для класса 3: 0.9325
ROC AUC для класса 4: 0.9600


Модель хуже справляется с средним сегментом зарплат (в районе 100-120 тысяч рублей) из-за того, что квантили зарплат находятся близко друг к другу по значениям (разница между 2,3,4 квантилями зарплат составляет всего около 20-35 тысяч, что близко, учитывая, что зарплаты разнятся от 40 до 600+ тысяч рублей). Но метрика тем не менее не слишком отличается для разных классов

Строим confusion matrix для понимания, какие классы модель путает с другими классами

In [ ]:
cm = confusion_matrix(y_val, y_pred_val)

heatmap = go.Heatmap(z=cm, x=['0', '1', '2', '3', '4'], y=['0', '1', '2', '3', '4'], colorscale='Blues', text = cm.tolist(), texttemplate="%{text}",
                    textfont={"size":20})

layout = go.Layout(title='Confusion Metrix')

fig = go.Figure(data=[heatmap], layout=layout)

fig.show()

Модель скорее чаще немного завышает зарплату, чем занижает и основная проблема с зарплатами в диапазоне от 80 до 160 тысяч рублей. Это пожет быть связано с тем, что модели сложно работать с такими профессиями, как программист, инженер, экономист и т.п., зарплаты которых могут разниться в широком диапазоне от области работы.

Обучим 5 регрессионных моделей, каждая из которых будет обучена на выборке лишь с одним предсказанном классом. 5 моделей нужно, чтобы каждый cat boost regressor был обучен под свой сегмент объектов, зарплаты которых лежат в определенном диапазоне (между значениями квантилей).

 Это будет намного эффективнее, чем просто запустить одну регрессионную модель для всей выборки, ведь каждая из пяти моделей будет специализироваться на своих классах объектов и соответственно настройка весов пяти моделей в совокупности будет более точной, чем у одной модели.

In [ ]:
regressors = {}

y_train_original = pd.Series(y_train_original)
y_val_original = pd.Series(y_val_original)

for quantile in range(5):
    mask_train = (y_pred_train == quantile)
    mask_val = (y_pred_val == quantile)

    mask_train = np.asarray(mask_train, dtype=bool).ravel()
    mask_val = np.asarray(mask_val, dtype=bool).ravel()

    X_train_q = X_train.loc[mask_train]
    y_train_q = y_train_original.loc[mask_train]

    X_val_q = X_val.loc[mask_val]
    y_val_q = y_val_original.loc[mask_val]
    print(f"Обучаем регрессор для квантиля {quantile}: {len(X_train_q)} примеров")

    reg = CatBoostRegressor(
        n_estimators=1000,
        learning_rate=1,
        max_depth=5,
        cat_features=cat_features,
        random_state=42,
        loss_function='RMSE',
        early_stopping_rounds=50,
        verbose=100
    )
    reg.fit(X_train_q, y_train_q, eval_set=(X_val_q, y_val_q))
    regressors[quantile] = reg


Обучаем регрессор для квантиля 0: 5173 примеров
0:	learn: 0.3960521	test: 0.3942385	best: 0.3942385 (0)	total: 48.4ms	remaining: 48.3s
100:	learn: 0.2221032	test: 0.2337132	best: 0.2315876 (99)	total: 2.35s	remaining: 20.9s
200:	learn: 0.1822527	test: 0.2099190	best: 0.2099190 (200)	total: 4.31s	remaining: 17.1s
300:	learn: 0.1619323	test: 0.1985776	best: 0.1985776 (300)	total: 5.71s	remaining: 13.3s
400:	learn: 0.1472615	test: 0.1910071	best: 0.1910071 (400)	total: 6.42s	remaining: 9.6s
500:	learn: 0.1371312	test: 0.1866709	best: 0.1866566 (497)	total: 7.13s	remaining: 7.1s
600:	learn: 0.1278543	test: 0.1856402	best: 0.1854575 (568)	total: 7.82s	remaining: 5.19s
700:	learn: 0.1192015	test: 0.1831349	best: 0.1828500 (680)	total: 9.31s	remaining: 3.97s
800:	learn: 0.1118188	test: 0.1802209	best: 0.1800609 (796)	total: 11.2s	remaining: 2.77s
900:	learn: 0.1058941	test: 0.1790968	best: 0.1790872 (899)	total: 13s	remaining: 1.42s
999:	learn: 0.1019082	test: 0.1784612	best: 0.1783940 (996)	

Предсказываем непрерывное значение зарплаты и получааем метрики качества модели

In [ ]:
def predict_salary(X, y_pred_class):
    y_pred_salary = np.zeros_like(y_pred_class, dtype=float)
    for quantile in range(len(regressors)):
        mask = (y_pred_class == quantile)
        if mask.sum() > 0:
            y_pred_salary[mask] = regressors[quantile].predict(X[mask])
    return y_pred_salary

y_train_original = np.exp(y_train_original.values.reshape(-1, 1)).ravel()
y_test_original = np.exp(y_test_original.reshape(-1, 1)).ravel()
y_val_original = np.exp(y_val_original.values.reshape(-1, 1)).ravel()

y_pred_train_salary = predict_salary(X_train, y_pred_train)
y_pred_val_salary = predict_salary(X_val, y_pred_val)
y_pred_test_salary = predict_salary(X_test, y_pred_test)

y_pred_train_salary  = np.exp(y_pred_train_salary.reshape(-1, 1)).ravel()
y_pred_test_salary = np.exp(y_pred_test_salary.reshape(-1, 1)).ravel()
y_pred_val_salary  = np.exp(y_pred_val_salary.reshape(-1, 1)).ravel()


def evaluate_predictions(y_true, y_pred, dataset_name="Dataset"):
    print(f"\n Оценка качества ({dataset_name}):")
    print(f" R²:   {r2_score(y_true, y_pred):.4f}")
    print(f" RMSE: {root_mean_squared_error(y_true, y_pred):.2f}")
    print(f" MAPE: {mean_absolute_percentage_error(y_true, y_pred) * 100:.2f}%")

evaluate_predictions(y_train_original, y_pred_train_salary, "Train")
evaluate_predictions(y_val_original, y_pred_val_salary, "Validation")
evaluate_predictions(y_test_original, y_pred_test_salary, "Test")


 Оценка качества (Train):
 R²:   0.7947
 RMSE: 25446.88
 MAPE: 14.03%

 Оценка качества (Validation):
 R²:   0.7885
 RMSE: 26094.63
 MAPE: 14.12%

 Оценка качества (Test):
 R²:   0.7333
 RMSE: 28234.99
 MAPE: 16.69%


Основные метрики для нас-это R² и MAPE. Показатель MAPE получился хорошим, учитывая, что изначально при небольших изменениях датасета модель показывала показатель MAPE = 50%+ по всем выборкам, R² показывает удовлетворительные значения. RMSE показывает, что на соответствующих выборках модель ошибается в среднем на данные значения

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [ ]:
pip install --upgrade --force-reinstall pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.45.1
    Uninstalling wheel-0.45.1:
      Successfully uninstalled wheel-0.45.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


Итоговый класс с сохранением тестовой выборки

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
import seaborn as sns
from sklearn.preprocessing import TargetEncoder
import sklearn
from catboost import CatBoostClassifier
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import plotly.graph_objs as go
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
class SalaryPredictor:

    def __init__(self):
        self.cat_features = ['premium', 'has_test', 'response_letter_required', 'archived', 'schedule', 'accept_temporary', 'accept_incomplete_resumes',
                'experience', 'employment', 'is_adv_vacancy' ,'employer_trusted', 'employer_type', 'metro_feature', 'has_email', 'industry', 'AO', 'exp_loc_category']
        self.cbс = CatBoostClassifier(
            n_estimators=2000,
            learning_rate=1,
            max_depth=5,
            cat_features=self.cat_features,
            random_state=42,
            eval_metric='MultiClass',
            loss_function='MultiClass',
            early_stopping_rounds=50,
            verbose=100
        )
        self.regressors = {}
        self.encoder_dict = {}
        self.target_encoder = TargetEncoder(categories = 'auto', smooth='auto')

    def save_model(self, filename="salary_model.pkl"):
        with open(filename, "wb") as f:
            pickle.dump(self, f)
        print(f"Модель сохранена в {filename}")

    @classmethod
    def load_model(cls, filename="salary_model.pkl"):
        with open(filename, "rb") as f:
            instance = pickle.load(f)
        print(f"Модель загружена из {filename}")
        return instance

    def preprocess(self, data):

            data = data.drop(data[data['salary'] > 800000].index.to_list(), axis = 0)
            #FEATURE ENGINERING

            #stations_within_km, distance_to_the_nearest(m) 	объедияем
            data['metro_feature'] = 'Никаких станций нет'

            data.loc[data['distance_to_the_nearest(m)'] > 1000, 'metro_feature'] = 'Станция далеко, но есть'

            data.loc[data['stations_within_km'] > 0, 'metro_feature'] = 'Станция рядом есть'

            #полная/неполная занятость:
            data.loc[data["employment"] != "Полная занятость", "employment"] = "Не полная занятость"

            #работодатель:
            data.loc[data["employer_type"] != "company", "employer_type"] = "not a company"

            #добавляю категорию включающую опыт+локацию:
            moscow_regions = ['свао', 'вао', 'цао', 'юзао', 'сзао', 'сао', 'ювао', 'новомосковский', 'зао', 'юао', 'троицкий', 'зелао']
            data['AO_loc'] = data['AO'].apply(
                lambda x: 'Москва' if x in moscow_regions else 'Не в Москве'
            )

            exp_values = data['experience'].unique()

            exp_categories = {exp_value: i+1 for i, exp_value in enumerate(exp_values)}

            data['exp_cat'] = data['experience'].map(exp_categories)

            ao_rules = {
                ('Москва', 1): 1,
                ('Москва', 2): 2,
                ('Москва', 3): 3,
                ('Москва', 4): 4,
                ('Не в Москве', 1): 5,
                ('Не в Москве', 2): 6,
                ('Не в Москве', 3): 7,
                ('Не в Москве', 4): 8
            }

            #создадим новую колонку с финальной категорией
            data['exp_loc_category'] = data.apply(lambda row: ao_rules[(row['AO_loc'], row['exp_cat'])], axis=1)
            #дропаем ненужные фичи
            data = data.drop(['lat','lon','stations_within_km', 'distance_to_the_nearest(m)', 'vacancy_type', 'exp_cat', 'AO_loc'], axis = 1)
            data = data.dropna()

            #нижний регистр
            data['schedule'] = data['schedule'].astype(str).str.lower()
            data['professional_roles'] = data['professional_roles'].astype(str).str.lower()
            data['experience'] = data['experience'].astype(str).str.lower()
            data['employment'] = data['employment'].astype(str).str.lower()
            data['industry'] = data['industry'].astype(str).str.lower()
            data['AO'] = data['AO'].astype(str).str.lower()

            #разделение данных
            X = data.drop('salary', axis = 1)
            y = data['salary']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
            X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, shuffle=True, random_state=42)


            #label encoder
            encoder_dict = {}
            for col in cat_features:
                encoder_dict[col] = preprocessing.LabelEncoder().fit(X_train[[col]])
                X_train[col] = encoder_dict[col].transform(X_train[[col]])
                X_test[col] = encoder_dict[col].transform(X_test[[col]])
                X_val[col] = encoder_dict[col].transform(X_val[[col]])



            encoder = TargetEncoder(categories = 'auto', smooth='auto')
            # Применение TargetEncoder
            X_train['professional_roles'] = X_train['professional_roles'].astype(str)
            X_test['professional_roles'] = X_test['professional_roles'].astype(str)

            encoder.fit(X_train[['professional_roles']], y_train)
            X_train['professional_roles'] = encoder.transform(X_train[['professional_roles']])
            X_test['professional_roles'] = encoder.transform(X_test[['professional_roles']])
            X_val['professional_roles'] = encoder.transform(X_val[['professional_roles']])


            scaler = preprocessing.StandardScaler().fit(X_train[['professional_roles']])
            X_train['professional_roles'] = scaler.transform(X_train[['professional_roles']])
            X_test['professional_roles'] = scaler.transform(X_test[['professional_roles']])
            X_val['professional_roles'] = scaler.transform(X_val[['professional_roles']])

            y_train = np.log(y_train.values.reshape(-1, 1)).flatten()
            y_test = np.log(y_test.values.reshape(-1, 1)).flatten()
            y_val = np.log(y_val.values.reshape(-1, 1)).flatten()

            y_train_original = y_train.copy()
            y_test_original = y_test.copy()
            y_val_original = y_val.copy()

            _, bins = pd.qcut(y_train, q=5, retbins=True, labels=False)

            y_train = pd.cut(y_train, bins=bins, labels=False, include_lowest=True)
            y_test = pd.cut(y_test, bins=bins, labels=False, include_lowest=True)
            y_val = pd.cut(y_val, bins=bins, labels=False, include_lowest=True)

            def type_converter(self, X):
                for i in cat_features:
                    X[i] = X[i].astype('category')
                return X
            X_train = type_converter(self, X_train)
            X_test = type_converter(self, X_test)
            X_val = type_converter(self, X_val)

            return X_train, X_test, X_val, y_train, y_test, y_val, y_train_original, y_test_original, y_val_original

    def fit(self, X_train, X_test, X_val, y_train, y_test, y_val, y_train_original, y_test_original, y_val_original):

        self.cbс.fit(X_train, y_train, eval_set=(X_val, y_val))

        y_pred_train = self.cbс.predict(X_train)
        y_pred_val = self.cbс.predict(X_val)

        y_train_original = pd.Series(y_train_original)
        y_val_original = pd.Series(y_val_original)

        for quantile in range(5):
            mask_train = (y_pred_train == quantile)
            mask_val = (y_pred_val == quantile)

            mask_train = np.asarray(mask_train, dtype=bool).ravel()
            mask_val = np.asarray(mask_val, dtype=bool).ravel()

            X_train_q = X_train.loc[mask_train]
            y_train_q = y_train_original.loc[mask_train]

            X_val_q = X_val.loc[mask_val]
            y_val_q = y_val_original.loc[mask_val]

            print(f"Обучаем регрессор для квантиля {quantile}: {len(X_train_q)} примеров")

            reg = CatBoostRegressor(
                n_estimators=1000,
                learning_rate=1,
                max_depth=5,
                cat_features=self.cat_features,
                random_state=42,
                loss_function='RMSE',
                early_stopping_rounds=50,
                verbose=100
            )

            reg.fit(X_train_q, y_train_q, eval_set=(X_val_q, y_val_q))
            self.regressors[quantile] = reg

    def predict(self, X, save_path=None):

        y_pred_class = self.cbс.predict(X)

        y_pred_salary = np.zeros_like(y_pred_class, dtype=float)

        for quantile in range(len(self.regressors)):
            mask = (y_pred_class == quantile)
            if mask.sum() > 0:
                y_pred_salary[mask] = self.regressors[quantile].predict(X[mask])

        y_pred_salary = np.exp(y_pred_salary)

        if save_path:
            pd.DataFrame(y_pred_salary, columns=["Predicted Salary"]).to_csv(save_path, index=False)
            print(f"Предсказания сохранены в {save_path}")

        return y_pred_salary


    def evaluate(self, X_train, X_val, X_test, y_train_original, y_val_original, y_test_original):
        y_pred_train_salary = self.predict(X_train)
        y_pred_val_salary = self.predict(X_val)
        y_pred_test_salary = self.predict(X_test, save_path="predictions_test.csv")

        y_train_original = np.exp(y_train_original.reshape(-1, 1)).ravel()
        y_val_original = np.exp(y_val_original.reshape(-1, 1)).ravel()
        y_test_original = np.exp(y_test_original.reshape(-1, 1)).ravel()

        self.evaluate_predictions(y_train_original, y_pred_train_salary, "Train")
        self.evaluate_predictions(y_val_original, y_pred_val_salary, "Validation")
        self.evaluate_predictions(y_test_original, y_pred_test_salary, "Test")

    @staticmethod
    def evaluate_predictions(y_true, y_pred, dataset_name="Dataset"):
        print(f"\nОценка качества ({dataset_name}):")
        print(f"R²:   {r2_score(y_true, y_pred):.4f}")
        print(f"RMSE: {root_mean_squared_error(y_true, y_pred):.2f}")
        print(f"MAPE: {mean_absolute_percentage_error(y_true, y_pred) * 100:.2f}%")


data = pd.read_csv('/content/drive/MyDrive/Datasets/vacancies.csv')
cat_features = ['premium', 'has_test', 'response_letter_required', 'archived', 'schedule', 'accept_temporary', 'accept_incomplete_resumes',
                'experience', 'employment', 'is_adv_vacancy' ,'employer_trusted', 'employer_type', 'metro_feature', 'has_email', 'industry', 'AO', 'exp_loc_category']

model = SalaryPredictor()

X_train, X_test, X_val, y_train, y_test, y_val, y_train_original, y_test_original, y_val_original = model.preprocess(data)

model.fit(X_train, X_test, X_val, y_train, y_test, y_val, y_train_original, y_test_original, y_val_original)

# Сохранение обученной модели
model.save_model("salary_model.pkl")

loaded_model = SalaryPredictor.load_model("salary_model.pkl")

# Использование загруженной модели для предсказаний
y_pred_test_salary = loaded_model.predict(X_test, save_path="predictions_test.csv")

# Предсказания для датасета
model.evaluate(X_train, X_val, X_test, y_train_original, y_val_original, y_test_original)

Предсказания сохранены в predictions_test.csv
Предсказания сохранены в predictions_test.csv

Оценка качества (Train):
R²:   0.7947
RMSE: 25446.88
MAPE: 14.03%

Оценка качества (Validation):
R²:   0.7885
RMSE: 26094.63
MAPE: 14.12%

Оценка качества (Test):
R²:   0.7333
RMSE: 28234.99
MAPE: 16.69%


In [ ]:
model_path = "/content/salary_model.pkl"

In [ ]:
import pickle

# Функция для загрузки модели
def load_model(model_path):
    with open(model_path, "rb") as f:
        return pickle.load(f)

# Загрузка модели
model = load_model(model_path)

In [ ]:
data_path = '/content/drive/MyDrive/Datasets/vacancies.csv' # Укажи путь к файлу
data = pd.read_csv(data_path)

In [ ]:
X_test_1 = model.preprocess(data)[1]

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was 

In [ ]:
y_pred_test_salary = model.predict(X_test_1, save_path="predictions_test.csv")

Предсказания сохранены в predictions_test.csv


In [ ]:
y_pred_test_salary

array([[203757.32595426],
       [ 59941.26629728],
       [ 72068.83990139],
       ...,
       [113695.78000342],
       [166812.51118443],
       [ 94687.94439403]])